## PROJETO PÁSCOA COM DADOS
#### Apoiei o Projeto Dado Humanizado do Prof. Cláudio Bonel. Foi uma iniciativa chamada "Páscoa com Dados", que entregou ovos de páscoa para 100 crianças do Complexo do Alemão/RJ em 2022.

In [1]:
# Importando as bibliotecas necessarias
import pandas as pd
from datetime import datetime
from pathlib import Path
import sqlite3 as sql

In [2]:
dtCarga = datetime.today().strftime('%d/%m/%Y %H:%M')
dtCarga

'14/04/2022 19:27'

### COLETANDO DADOS DOS BATALHÕES DE POLÍCIA MILITAR (BPM)

In [3]:
# Definindo o endereço físico do arquivo CSV
LocalArquivo = "E:\\Projetos\\Python\\08 - Prof Bonel\\Python Integracao Dados com SQL\\"

# Criar Data Frame com os dados dos BPMs
tbLogBPM = pd.DataFrame(pd.read_excel(LocalArquivo + "BPM.xlsx"))

# Criando coluna com a data da carga
tbLogBPM['dtCarga'] = dtCarga

In [4]:
tbLogBPM.head(10)

,COD_BPM,NM_BPM,Endereco,dtCarga
0,2,2º BPM,"R. Álvaro Ramos, 155",14/04/2022 19:27
1,3,3º BPM,"R. Lucídio Lago, 181",14/04/2022 19:27
2,4,4º BPM,"R. Francisco Eugênio, 228",14/04/2022 19:27
3,5,5º BPM,"Praça Cel. Assunção, S/N",14/04/2022 19:27
4,6,6º BPM,"R. Barão de Mesquita, 625",14/04/2022 19:27
5,7,7º BPM,"R. Dr. Alfredo Backer, 367",14/04/2022 19:27
6,8,8º BPM,"Rua Tenente Coronel Cardoso, s/n",14/04/2022 19:27
7,9,9º BPM,"R. Tacaratu, 94",14/04/2022 19:27
8,10,10º BPM,"Rod. Lúcio Meira, 47000",14/04/2022 19:27
9,11,11º BPM,"R. Voluntários da Pátria, 474",14/04/2022 19:27


In [5]:
# dimensao
tbLogBPM.shape

(41, 4)

In [6]:
# Informações
tbLogBPM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   COD_BPM   41 non-null     int64 
 1   NM_BPM    41 non-null     object
 2   Endereco  41 non-null     object
 3   dtCarga   41 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.4+ KB


In [7]:
# Verificando a existenecia de nulos
tbLogBPM.isnull().sum()

COD_BPM     0
NM_BPM      0
Endereco    0
dtCarga     0
dtype: int64

### CONECTANDO NO DW E INFORMANDO AO PYTHON QUE USAREI LINGUAGEM SQL

In [25]:
# Manipulando o sistema de arquivos no Windows
endereco = Path('E:\\Projetos\\Python\\08 - Prof Bonel\\Python Integracao Dados com SQL\\')

BDODS = endereco / "pascoaODS.db"
BDDW = endereco / "pascoaDW.db"

# Conectando no BDODS
conexaoODS = sql.connect(BDODS)

# Definição de manipulação de dados em SQL. Métdo cursor permite manipular dados no SQL
SQL_ODS = conexaoODS.cursor()

### CONSTRUINDO A TABELA DE LOG DE REGISTROS DO BPM (tbLogBPM), no ODS

In [9]:
# Definir a query de criação da tabela
qry_tbLogBPM = '''
    
    CREATE TABLE IF NOT EXISTS tbLogBPM (
    
    codBPM INTEGER,
    nmBPM VARCHAR (7),
    enderecoBPM VARCHAR (200),
    dtCarga DATETIME

    )
'''

# executar a criação da tbLogBPM
SQL_ODS.execute(qry_tbLogBPM)

# definição da query de criação de index
qry_idx_tbLogBPM_codBPM = "CREATE INDEX IF NOT EXISTS idx_tbLogBPM_codBPM ON tbLogBPM (codBPM)"

# executar a criação do index
SQL_ODS.execute(qry_idx_tbLogBPM_codBPM)

### INSERINDO DADOS DE BPM NA TABELA tbLogBPM

In [11]:
# Inserindo registros na tabela tbLogBPM
print("Iniciando a inserção de dados na tabela tbLogBPM.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogBPM (codBPM,nmBPM,enderecoBPM,dtCarga) VALUES (?,?,?,?)''',tbLogBPM.values.tolist())

# confirmar a transação
conexaoODS.commit()

# Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogBPM), "registros inseridos na tbLogBPM!")

Iniciando a inserção de dados na tabela tbLogBPM.
-------------------------------------------------
Carga finalizada! 41 registros inseridos na tbLogBPM!


### COLETANDO DADOS DA ÁREA DOS BATALHÕES DE POLÍCIA MILITAR (BPM)

In [12]:
 #Criar DataFrame com os dados dos BPMs
tbLogAreaBPM = pd.DataFrame(pd.read_csv(LocalArquivo + "areaBPMv1.csv"))

#Criando coluna com a data da carga
tbLogAreaBPM['dtCarga'] = dtCarga

In [13]:
tbLogAreaBPM.head()

,aisp,area_aisp_km2,dtCarga
0,5,15.4,14/04/2022 19:27
1,4,17.1,14/04/2022 19:27
2,2,15.0,14/04/2022 19:27
3,23,25.2,14/04/2022 19:27
4,19,5.1,14/04/2022 19:27


In [14]:
# dimensão
tbLogAreaBPM.shape

(39, 3)

In [15]:
# Informações
tbLogAreaBPM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   aisp           39 non-null     int64  
 1   area_aisp_km2  39 non-null     float64
 2   dtCarga        39 non-null     object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.0+ KB


In [16]:
# Verificando existencia de nulos
tbLogAreaBPM.isnull().sum()

aisp             0
area_aisp_km2    0
dtCarga          0
dtype: int64

### CONSTRUINDO A TABELA DE LOG DE REGISTROS DA ÁREA DOS BPMs (tbLogAreaBPM), no ODS

In [17]:
# Definir a query de criação da tabela
qry_tbLogAreaBPM = '''
    
    CREATE TABLE IF NOT EXISTS tbLogAreaBPM (
    
    codBPM INTEGER,
    areaBPM REAL (5,2),
    dtCarga DATETIME

    )
'''

# executar a criação da tbLogAreaBPM
SQL_ODS.execute(qry_tbLogAreaBPM)

# definição da query de criação de index
qry_idx_tbLogAreaBPM_codBPM = "CREATE INDEX IF NOT EXISTS idx_tbLogAreaBPM_codBPM ON tbLogAreaBPM (codBPM)"

#executar a criação do index
SQL_ODS.execute(qry_idx_tbLogAreaBPM_codBPM)

### INSERINDO DADOS DA ÁREA DOS BPMs NA TABELA tbLogAreaBPM

In [18]:
# Inserindo registros na tabela tbLogAreaBPM
print("Iniciando a inserção de dados na tabela tbLogAreaBPM.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogAreaBPM (codBPM,areaBPM,dtCarga) VALUES (?,?,?)''',tbLogAreaBPM.values.tolist())

# confirmar a transação
conexaoODS.commit()

# Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogAreaBPM), "registros inseridos na tbLogAreaBPM!")

Iniciando a inserção de dados na tabela tbLogAreaBPM.
-------------------------------------------------
Carga finalizada! 39 registros inseridos na tbLogAreaBPM!


### CRIANDO O DATAFRAME COM O SELECT DE CRIAÇÃO DA DIMENSÃO DP (dBPM), NO DW

In [26]:
# Definir a query de criação da dimensão dBPM

qry_dBPM = '''
    SELECT
        codBPM,
        nmBPM,
        enderecoBPM,
        areaBPM
    FROM 
    (
        SELECT 
            a.codBPM,
            a.nmBPM,
            a.enderecoBPM,
            b.areaBPM,
            max(a.dtCarga)
        FROM tbLogBPM a
        JOIN tbLogAreaBPM b
        ON a.codBPM = b.codBPM
        WHERE a.dtCarga = (
                            SELECT MAX(x.dtCarga)
                            FROM tbLogBPM x
                       )
        GROUP BY
            a.codBPM,
            a.nmBPM,
            a.enderecoBPM,
            b.areaBPM
    ) a
'''

dBPM = pd.read_sql(qry_dBPM,conexaoODS)

### FECHANDO A CONEXÃO COM O ODS

In [27]:
# Confirmar a transação
conexaoODS.commit()

# Fechar a conexão com o pascoaODS
conexaoODS.close()

### CONECTANDO NO DW E INFORMANDO AO PYTHON QUE USAREI LINGUAGEM SQL

In [21]:
# Conectar no BDODS
conexaoDW = sql.connect(BDDW)

# Definição de manipulação de dados em SQL
SQL_DW = conexaoDW.cursor()

### CONSTRUINDO A TABELA DE DIMENSÃO BPM (dBPM), no DW

In [22]:
#Definir a query de criação da tabela
qry_dBPM = '''
    
    CREATE TABLE IF NOT EXISTS dBPM (
    
    idBPM INTEGER PRIMARY KEY AUTOINCREMENT,
    codBPM INTEGER,
    nmBPM VARCHAR (7),
    enderecoBPM VARCHAR (200),
    areaBPM REAL (5,2)

    )
'''

#bexecutar a criação da tbLogBPM
SQL_DW.execute(qry_dBPM)

#bexecutar a criação do index idBPM
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_dBPM_idBPM ON dBPM (idBPM)")

#bexecutar a criação do index codBPM
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_dBPM_codBPM ON dBPM (codBPM)")

### DELETANDO DADOS ANTIGOS E REINICIANDO AO AUTOINCREMENTAL DA CHAVE, DA TABELA dBPM

In [23]:
# Deletar os dados atuais
SQL_DW.execute("DELETE FROM dBPM")

# Reiniciando o autoincramental da chave
SQL_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dBPM'")

# Confirmar a transação
conexaoDW.commit()

### INSERINDO DADOS NA DIMENSÃO BPM (dBPM), NO DW

In [28]:
# Inserindo registros na tabela dBPM
print("Iniciando a inserção de dados na tabela dBPM.")
print("-------------------------------------------------")

SQL_DW.executemany('''INSERT INTO dBPM (codBPM,nmBPM,enderecoBPM,areaBPM) VALUES (?,?,?,?)''',dBPM.values.tolist())

# confirmar a transação
conexaoDW.commit()

# Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dBPM), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela dBPM.
-------------------------------------------------
Carga finalizada! 39 registros inseridos na tbLogDP!


### FECHANDO A CONEXÃO COM O DW

In [29]:
# Confirmar a transação
conexaoDW.commit()

# Fechar a conexão
conexaoDW.close()

In [30]:
!ls

01 - WebScrapingDadosIBGEdoRJ.ipynb
02 - Tabela tbLogMunic criada para manter o historico.png
03 - Tabela dMunicipio criada para manter os dados atualizados.png
04 - CargaDimensaoDelegaciasPolicia.ipynb
05 - IntegrandoDadosIBGE_DP_BPM.ipynb
BPM.xlsx
DP.csv
ResponsavelDP.csv
areaBPMv1.csv
pascoaDW.db
pascoaODS.db
popDP.csv
